In [1]:
import pandas as pd
import os

In [9]:
df = pd.read_csv('data/complete_min_data.csv')

In [10]:
df.head()

,Unnamed: 0,GEO_ID,NAME,full_name,st,pop2018,lat,lng,state,Zhvi_1bedroom,Zhvi_2bedroom,Zhvi_3bedroom,Zhvi_4bedroom,Zhvi_5Bedroom
0,0,1600000US0167056,"Russellville city, Alabama",Russellville AL,AL,9711,34.5056,-87.7282,Alabama,False,True,True,True,True
1,1,1600000US0178984,"Vina town, Alabama",Vina AL,AL,363,34.3757,-88.0602,Alabama,False,False,False,False,False
2,2,1600000US0162688,"Providence town, Alabama",Providence AL,AL,179,32.3424,-87.7761,Alabama,False,False,False,False,False
3,3,1600000US0131096,"Grant town, Alabama",Grant AL,AL,933,34.5189,-86.2492,Alabama,False,True,True,True,True
4,4,1600000US0119816,"Daviston town, Alabama",Daviston AL,AL,186,33.0557,-85.6371,Alabama,False,False,False,False,False


In [12]:
wy = df[(df.st == 'WY')]

In [14]:
wy.groupby('Zhvi_3bedroom')['Zhvi_3bedroom'].count()

Zhvi_3bedroom
False     94
True     106
Name: Zhvi_3bedroom, dtype: int64

In [15]:
wy_zip = pd.read_csv('data/housing/bedroomziplatlng/Zip_Zhvi_3bedroom_zip.csv')

In [17]:
wy_zip.head()

,Unnamed: 0,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,2008-01,2008-02,...,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,lat,lng
0,0,61639,10025,New York,NY,New York-Newark-Jersey City,New York County,1,NaN,NaN,...,2.042780e+06,2.035366e+06,2.034423e+06,2.025748e+06,2.005972e+06,1.973243e+06,1.958949e+06,1.948357e+06,40.798502,-73.96811
1,1,84654,60657,Chicago,IL,Chicago-Naperville-Elgin,Cook County,2,NaN,NaN,...,5.917907e+05,5.927763e+05,5.910983e+05,5.901127e+05,5.871780e+05,5.847233e+05,5.823167e+05,5.815510e+05,41.940832,-87.65852
2,2,61637,10023,New York,NY,New York-Newark-Jersey City,New York County,3,NaN,NaN,...,3.422120e+06,3.384821e+06,3.368683e+06,3.347543e+06,3.338804e+06,3.334868e+06,3.346086e+06,3.355841e+06,40.776099,-73.98285
3,3,91982,77494,Katy,TX,Houston-The Woodlands-Sugar Land,Harris County,4,197135.0,196620.0,...,2.540797e+05,2.533590e+05,2.528003e+05,2.530083e+05,2.531913e+05,2.541217e+05,2.546267e+05,2.555150e+05,29.760833,-95.81104
4,4,84616,60614,Chicago,IL,Chicago-Naperville-Elgin,Cook County,5,NaN,NaN,...,6.981180e+05,6.997963e+05,6.988157e+05,6.963590e+05,6.934267e+05,6.905770e+05,6.891443e+05,6.878177e+05,41.922682,-87.65432


In [18]:
wy_zip = wy_zip[wy_zip.State == 'WY']

In [20]:
wy_zip.shape

(124, 153)

In [21]:
wy = wy[wy.Zhvi_3bedroom == False]

In [59]:
wy_zip.iloc[2]['2019-11']

287674.666666667

In [23]:
from math import cos, asin, sqrt

def distance(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p)*cos(lat2*p) * (1-cos((lon2-lon1)*p)) / 2
    return 12742 * asin(sqrt(a))

In [24]:
lat = wy_zip.lat.values
lng = wy_zip.lng.values

In [27]:
clat = wy.lat.values
clng = wy.lng.values

In [26]:
import numpy as np

#index_min = np.argmin(values)

In [66]:
mapper = []
for i in range(len(clat)):
    inlat = clat[i]
    inlng = clng [i]
    mem = []
    for a in range(len(lat)):
        res = distance(lat[a], lng[a], inlat, inlng)
        mem.append(res)
    dist = min(mem)
    index_min = np.argmin(mem)
    city = wy_zip.iloc[index_min]['City']
    price = wy_zip.iloc[index_min]['2019-11']
    mapper.append([inlat, inlng, lat[index_min], lng[index_min], index_min, city, str(price), dist])
    
    

In [68]:
distdf = pd.DataFrame(mapper, columns=['clat', 'clng', 'zlat', 'zlng', 'zindex','city','price', 'distance'])

In [64]:
distdf.head()

,clat,clng,zlat,zlng,zindex,city,price,distance
0,41.7761,-110.5555,41.399751,-110.36624,33,Lyman,206804.0,44.710326
1,41.0346,-107.5357,41.475612,-106.75888,34,Saratoga,200693.0,81.372965
2,41.0345,-107.6574,41.475612,-106.75888,34,Saratoga,200693.0,89.707235
3,42.7612,-104.4583,42.507335,-104.42420,114,Jay Em,496897.0,28.365997
4,42.7652,-104.9255,42.792664,-104.94156,122,Manville,377588.0,3.323262


In [39]:
distdf.shape

(94, 6)

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(go.Scattermapbox(
    mode = "markers",
    text = distdf.city.values,
    lon = distdf.clng.values,
    lat = distdf.clat.values,
    marker = {'size': 10}))


fig.add_trace(go.Scattermapbox(
    mode = "markers",

    hovertext = distdf.price.values,
    lon = distdf.zlng.values,
    lat = distdf.zlat.values,
    marker = {'size': 10}))

fig.update_layout(
    margin ={'l':0,'t':0,'b':0,'r':0},
    mapbox = {
        'center': {'lon': 10, 'lat': 10},
        'style': "stamen-terrain",
        'center': {'lon': -20, 'lat': -20},
        'zoom': 1})

fig.show()

<img src="https://raw.githubusercontent.com/Lambda-School-Labs/city-data-comparison-ds/master/images/wy_zip_lat_long_mapping.png">

In [86]:
threebed = pd.read_csv('data/housing/bedroomziplatlng/Zip_Zhvi_3bedroom_zip.csv')
onebed = pd.read_csv('data/housing/bedroomziplatlng/Zip_Zhvi_1bedroom_zip.csv')

In [87]:
threebed.shape

(24076, 153)

In [88]:
onebed.shape

(13607, 153)